In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

2023-01-06 05:53:14.406362: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 05:53:14.490118: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-06 05:53:15.044075: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-06 05:53:15.044126: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
__name__ = "RNN"

# Model

In [3]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


In [4]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [5]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

FileNotFoundError: [Errno 2] No such file or directory: 'data/gestures/left click/51/0.npy'

In [ ]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = getModel()
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

2023-01-05 20:47:02.474686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/1000
2/2 [==============================] - 4s 36ms/step - loss: 0.6957 - categorical_accuracy: 0.3860
Epoch 2/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.6939 - categorical_accuracy: 0.4035
Epoch 3/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.6908 - categorical_accuracy: 0.6140
Epoch 4/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.6889 - categorical_accuracy: 0.7018
Epoch 5/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.6869 - categorical_accuracy: 0.6667
Epoch 6/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.6850 - categorical_accuracy: 0.6491
Epoch 7/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.6743 - categorical_accuracy: 0.8772
Epoch 8/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.6642 - categorical_accuracy: 0.6140
Epoch 9/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.6354 - categorical_accuracy: 0.6491
E

In [ ]:

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 203,426
Trainable params: 203,426
Non-trai

In [ ]:
model.save(f"{modelPath}/{__name__}/test.h5")